website: https://www.freecodecamp.org/news/how-to-build-a-movie-recommendation-system-based-on-collaborative-filtering/

IMPorting libraries

In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors


Ratings file into Pandas dataframes and look at the columns

In [9]:
user_ratings_df = pd.read_csv("ratings.csv")
user_ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


MOvie meta data

In [10]:
movie_metadata = pd.read_csv("movies.csv")
movie_metadata = movie_metadata[['movieId','title', 'genres']]
movie_metadata.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


comnbining meta datas

In [11]:
movie_data = user_ratings_df.merge(movie_metadata, on='movieId')
movie_data.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


Making User iD matrix

In [12]:

user_item_matrix = user_ratings_df.pivot(index=['userId'], columns=['movieId'], values='rating').fillna(0)
user_item_matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


defining KNN model

In [13]:
# Define a KNN model on cosine similarity
cf_knn_model= NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)


# Fitting the model on our matrix
cf_knn_model.fit(user_item_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=10)

a function to provide the desired number of movie recommendations, given a movie title as input

1 n_recs: Controls the number of final recommendations that we would get as output
2 Movie_name: Input movie name, based on which we find new recommendations
3 Matrix: The User-Movie Rating matrix

In [14]:
def movie_recommender_engine(movie_name, matrix, cf_model, n_recs):
    # Extract input movie ID
    movie_id = movie_metadata[movie_metadata['title'] == movie_name]['movieId'].values[0]
    
    if movie_id not in matrix.index:
        print(f"Error: The movie '{movie_name}' was not found in the user-item matrix.")
        return None
    # Calculate neighbour distances
    distances, indices = cf_model.kneighbors(matrix.loc[movie_id].values.reshape(1, -1), n_neighbors=n_recs)
    movie_rec_ids = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
    
    # List to store recommendations
    cf_recs = []
    for i in movie_rec_ids:
        cf_recs.append({'Title':movie_metadata['title'][i[0]],'Distance':i[1]})
    
    # Select top number of recommendations needed
    if len(cf_recs) >= n_recs:
        df = pd.DataFrame(cf_recs, index = range(1,n_recs+1))
    else:
        print(f"Only {len(cf_recs)} recommendations could be generated for '{movie_name}'.")
        df = pd.DataFrame(cf_recs)
    
    return df

OUTPUT

In [16]:
n_recs = 10
movie_recommender_engine('Pulp Fiction (1994)', user_item_matrix, cf_knn_model, n_recs)

Only 9 recommendations could be generated for 'Pulp Fiction (1994)'.


,Title,Distance
0,Flipper (1996),0.703944
1,Sense and Sensibility (1995),0.702204
2,Pretty Woman (1990),0.701646
3,In the Realm of the Senses (Ai no corrida) (1976),0.682915
4,Lamerica (1994),0.676956
5,Speechless (1994),0.674799
6,Moll Flanders (1996),0.669055
7,Only You (1994),0.643052
8,Double Happiness (1994),0.634758
